## Importing DataSet From Kaggle

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'newspaper-text-summarization-cnn-dailymail' dataset.
Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


In [8]:
import torch
import numpy as np
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

## Dataset Preparation

In [9]:
dataset= load_dataset("csv", data_files='/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
full_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

In [11]:
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

In [12]:
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['id', 'article', 'highlights'],
    num_rows: 229690
})
Dataset({
    features: ['id', 'article', 'highlights'],
    num_rows: 57423
})


## Analysis

In [13]:
def find_longest_length(dataset):
    """
    Find the longest article and summary in the entire training set.
    """
    max_length = 0
    counter_4k = 0
    counter_2k = 0
    counter_1k = 0
    counter_500 = 0
    for text in dataset:
        corpus = [
            word for word in text.split()
        ]
        if len(corpus) > 4000:
            counter_4k += 1
        if len(corpus) > 2000:
            counter_2k += 1
        if len(corpus) > 1000:
            counter_1k += 1
        if len(corpus) > 500:
            counter_500 += 1
        if len(corpus) > max_length:
            max_length = len(corpus)
    return max_length, counter_4k, counter_2k, counter_1k, counter_500

longest_article_length, counter_4k, counter_2k, counter_1k, counter_500 = find_longest_length(dataset['article'])
print(f"Longest article length: {longest_article_length} words")
print(f"Artciles larger than 4000 words: {counter_4k}")
print(f"Artciles larger than 2000 words: {counter_2k}")
print(f"Artciles larger than 1000 words: {counter_1k}")
print(f"Artciles larger than 500 words: {counter_500}")
longest_summary_length, counter_4k, counter_2k, counter_1k, counter_500 = find_longest_length(dataset['highlights'])
print(f"Longest summary length: {longest_summary_length} words")
print(f"Summaries larger than 4000 words: {counter_4k}")
print(f"Summaries larger than 2000 words: {counter_2k}")
print(f"Summaries larger than 1000 words: {counter_1k}")
print(f"Summaries larger than 500 words: {counter_500}")

Longest article length: 2347 words
Artciles larger than 4000 words: 0
Artciles larger than 2000 words: 12
Artciles larger than 1000 words: 48875
Artciles larger than 500 words: 193363
Longest summary length: 1296 words
Summaries larger than 4000 words: 0
Summaries larger than 2000 words: 0
Summaries larger than 1000 words: 3
Summaries larger than 500 words: 12


In [14]:
def find_avg_sentence_length(dataset):
    """
    Find the average sentence in the entire training set.
    """
    sentence_lengths = []
    for text in dataset:
        corpus = [
            word for word in text.split()
        ]
        sentence_lengths.append(len(corpus))
    return sum(sentence_lengths)/len(sentence_lengths)

avg_article_length = find_avg_sentence_length(dataset_train['article'])
print(f"Average article length: {avg_article_length} words")
avg_summary_length = find_avg_sentence_length(dataset_train['highlights'])
print(f"Averrage summary length: {avg_summary_length} words")

Average article length: 692.2672950498498 words
Averrage summary length: 51.605028516696414 words


## Configurations

In [15]:
MODEL = 't5-base'
BATCH_SIZE = 4
NUM_PROCS = 4 # Parallel Data Loading
EPOCHS = 10
OUT_DIR = 'results_t5base'
MAX_LENGTH = 512 # Maximum context length to consider while preparing dataset.

## Tokenizer & Tokenization

In [16]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [17]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['article']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['highlights']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the function to the whole dataset
tokenized_train = dataset_train.select(range(2000)).map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.select(range(400)).map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

## Model

In [18]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

222,903,552 total parameters.
222,903,552 training parameters.


## Rouge Metric

In [19]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4dc79e9f1a685931dc4d359597a466ef1a2094fc450183c1d0b04d54639afdd8
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [20]:
import evaluate
rouge = evaluate.load("rouge")

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [22]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

## Training

In [37]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_steps=200,
    save_strategy='epoch',
    eval_strategy="epoch",
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=2
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

history = trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
1,0.133000,0.237526,0.622600,0.366700,0.582900,75.842500
2,0.182200,0.229991,0.623100,0.364600,0.581300,75.845000
3,0.196500,0.234091,0.621400,0.361900,0.578900,75.845000
4,0.138600,0.236512,0.620000,0.362700,0.579000,75.845000


In [38]:
tokenizer.save_pretrained(OUT_DIR)

('results_t5base/tokenizer_config.json',
 'results_t5base/special_tokens_map.json',
 'results_t5base/spiece.model',
 'results_t5base/added_tokens.json')

In [39]:
!zip -r {OUT_DIR} {OUT_DIR}

  adding: results_t5base/ (stored 0%)
  adding: results_t5base/spiece.model (deflated 48%)
  adding: results_t5base/special_tokens_map.json (deflated 85%)
  adding: results_t5base/checkpoint-2000/ (stored 0%)
  adding: results_t5base/checkpoint-2000/optimizer.pt (deflated 8%)
  adding: results_t5base/checkpoint-2000/trainer_state.json (deflated 83%)
  adding: results_t5base/checkpoint-2000/scheduler.pt (deflated 62%)
  adding: results_t5base/checkpoint-2000/config.json (deflated 63%)
  adding: results_t5base/checkpoint-2000/model.safetensors (deflated 8%)
  adding: results_t5base/checkpoint-2000/rng_state.pth (deflated 26%)
  adding: results_t5base/checkpoint-2000/generation_config.json (deflated 29%)
  adding: results_t5base/checkpoint-2000/training_args.bin (deflated 53%)
  adding: results_t5base/tokenizer_config.json (deflated 94%)
  adding: results_t5base/events.out.tfevents.1762256530.f4094f19ba70.561.0 (deflated 68%)
  adding: results_t5base/added_tokens.json (deflated 83%)
  add

## Inference

In [27]:
# Download data.
!wget "https://www.dropbox.com/scl/fi/561r8pfhem4lu70hf438q/inference_data.zip?rlkey=aedt2saqmmp3a67qc4o34k04y&dl=1" -O inference_data.zip

--2025-11-04 12:06:01--  https://www.dropbox.com/scl/fi/561r8pfhem4lu70hf438q/inference_data.zip?rlkey=aedt2saqmmp3a67qc4o34k04y&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc41717489afae0161df04119fae.dl.dropboxusercontent.com/cd/0/inline/C0jwUabGop9NXtj03Cuv48wMsuL3TeYqEEifr7vZh5A9U1m7xgddr0ZypGCXQf_YgntcJwmfEMberPJ9AgM-ZnCbsOdriSYP77LLDZcZVlm7DfL0HNvIv7uGXnavsNH5urNXNK1Z6h6CoUpGruKv20F0/file?dl=1# [following]
--2025-11-04 12:06:01--  https://uc41717489afae0161df04119fae.dl.dropboxusercontent.com/cd/0/inline/C0jwUabGop9NXtj03Cuv48wMsuL3TeYqEEifr7vZh5A9U1m7xgddr0ZypGCXQf_YgntcJwmfEMberPJ9AgM-ZnCbsOdriSYP77LLDZcZVlm7DfL0HNvIv7uGXnavsNH5urNXNK1Z6h6CoUpGruKv20F0/file?dl=1
Resolving uc41717489afae0161df04119fae.dl.dropboxusercontent.com (uc41717489afae0161df04119fae.dl.dropboxusercontent.com)...

In [28]:
!unzip inference_data.zip

Archive:  inference_data.zip
  inflating: inference_data/file_1.txt  
  inflating: inference_data/file_2.txt  


In [29]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

import glob

In [40]:
model_path = f"{OUT_DIR}/checkpoint-2000"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(OUT_DIR)

In [41]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=50,
        num_beams=num_beams,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [42]:
import pprint as pp
for file_path in glob.glob('inference_data/*.txt'):
    file = open(file_path)
    text = file.read()
    summary = summarize_text(text, model, tokenizer)
    pp.pprint(summary)
    print('-'*75)

('The artificial intelligence industry was shaken by tectonic shifts over the '
 'weekend . Sam Altman, the leader of one of the world’s most influential AI '
 'companies, was fired Friday night by the startup’s board in')
---------------------------------------------------------------------------
('Microsoft has hired Sam Altman to lead its innovation in artificial '
 'intelligence . The co-founder of OpenAI was ousted as CEO on Friday . Emmett '
 'Shear, the former CEO of Amazon’s streaming service Twitch')
---------------------------------------------------------------------------


In [43]:
while True:
    text = input("\nEnter text to summarize (or type 'exit' to quit): ")
    if text.lower() == 'exit':
        break

    summary = summarize_text(text, model, tokenizer)
    print("\nSummary:", summary)


Enter text to summarize (or type 'exit' to quit): Artificial Intelligence (AI) has rapidly evolved over the past decade, transforming industries and daily life. From virtual assistants like Siri and Alexa to self-driving cars and advanced medical diagnostics, AI is now an essential part of modern technology. Machine learning, a subset of AI, enables systems to learn from data and improve performance without explicit programming. However, with these advancements come challenges related to privacy, ethics, and job displacement. Balancing innovation with responsibility remains a crucial goal for researchers and policymakers around the world.

Summary: Artificial Intelligence (AI) has rapidly evolved over the past decade . With advancements come challenges related to privacy, ethics, and job displacement . Balancing innovation with responsibility remains a crucial goal for researchers .

Enter text to summarize (or type 'exit' to quit): exit
